# Script to match a Spotify song with a Youtube video

* Data needs to be in CSV format
* Artist represents a list of all the artists contributing to that song
* In this example I use the video matching to get the view count

In [16]:
from __future__ import print_function    # (at top of module)
import warnings
#warnings.filterwarnings('always')
from spotipy.oauth2 import SpotifyClientCredentials
import json
import spotipy
import time
import sys
import csv
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
% matplotlib inline
plt.rcParams['figure.figsize'] = [10, 10]
from matplotlib.pyplot import figure
import math
import seaborn as sns
import config
import requests
import urllib

fileName = "google_cleaned_data.csv"

In [17]:
data = pd.read_csv(fileName ,encoding='utf-8')
data = data.drop_duplicates(subset=['song_id'], keep='first')
#data = data.drop_duplicates(subset=['song_title'], keep='first')
# delete all rows with column 'Age' has value 30 to 40 
#indexNames = data[ (data['popularity'] < 50) | (data['total_no_streams'] < 1000000) ].index
#data.drop(indexNames , inplace=True)
print("Number of entries in original data after cleaning: " + str(len(data.index)))

Number of entries in original data after cleaning: 3094


In [20]:
# DEBUGGING : print statements are commented out

# Build the query from the data
temp_query = ""
youtube_view_counts = []
youtube_video_titles = []
total_data_length = str(len(data.index))
cur_index = 1

# For each row in the dataframe
for index, row in data.iterrows():
    
    # Progress indicator
    print(cur_index, "/", total_data_length)
    cur_index += 1
    
    # Build the query as song_title + artists (from the Spotify API)
    temp_query += row['song_title'].replace(',', "") + " " + row['artist'].replace('[', "").replace(']', "").replace(',', "")
    
    #print("===============")
    #print(temp_query)
    
    params_for_query = {"part": "snippet",
                        "maxResults": 5,
                        "order": "relevance",
                        "pageToken": "",
                        "q": temp_query,
                        "key": config.youtube_client_secret,
                        "type": "video",
                        }
    url1 = "https://www.googleapis.com/youtube/v3/search"
    
    # Get top 5 video results ordered by relevance
    # TODO: Still some matching problems - how to get the most relevant video with the most views from official sources?
    page_query = requests.request(method="get", url=url1, params=params_for_query)
    j_results_query = json.loads(page_query.text)
    
    #print("QUERY ============================")
    
    max_viewcount = 0
    max_viewcount_title = ""
    
    # Iterate through the first 5 results to find the relevant video with most views
    for item in j_results_query['items']:
    
        
        #print("Video title: ", item['snippet']['title'])
        
        # Video title of a video to match the query string
        video_title = item['snippet']['title']

        
        #print("Video id: ", item['id']['videoId'])
        
        # Get the video ID of a video to match the query string
        video_id = item['id']['videoId']

        # Next, using this video ID we need to get the view count of that video
        params_for_stats = {"part": "statistics",
                            "id": video_id,
                            "key": config.youtube_client_secret,
                           }
        url2 = "https://www.googleapis.com/youtube/v3/videos"
        page_stats = requests.request(method="get", url=url2, params=params_for_stats)
        j_results_stats = json.loads(page_stats.text)

        # Check if problem with retrieving statistics
        if 'items' in j_results_stats:
            
            #print("Video view count: ", j_results_stats['items'][0]['statistics']['viewCount'])
            
            # Get the view count
            view_count  = j_results_stats['items'][0]['statistics']['viewCount']
            
            # This is my hacky matching condition
            # The video we want to match to our spotify song has to be the version of the song with the most views
            # BUT we also need to make sure that it is the same song
            # The .split() gets rid of some useless parts in a song title from spotify such as this
            # e.g. Spotify title = "Eastside (feat. Elton John)" - > "Eastside"
            # e.g. If we want the matching song for "Ariana Grande - thank u, next" and the results from the YT API are 
            #           1. "Ariana Grande - thank u, next (1 million views)"
            #           2. "Ariana Grande - breathin (5 million views)"
            # We would be tempted to just take the one with more views but in fact we need to double check so we only update the max
            # if the string "thank u, next" is also in the youtube video title
            # Hope this makes sense :) 
            if int(view_count) > max_viewcount and row['song_title'].split('(', 1)[0].split('-', 1)[0].rstrip().lower() in video_title.lower():
                max_viewcount = int(view_count)
                max_viewcount_title = video_title
            
    #print("CHOICE IS: ", max_viewcount_title, " ---- ", max_viewcount)
    
    # Keep the counts in a list
    youtube_view_counts.append(int(max_viewcount))
    
    #Keep titles in a list
    # Commas in the title would interefere with the CSV format so we get rid of them, also [] because they are list notation
    youtube_video_titles.append(max_viewcount_title.replace(",", "").replace('[','(').replace(']', ')'))
    
    temp_query = ""
    
# Add the two new columns to the data
data['youtube_view_count'] = youtube_view_counts
data['youtube_video_title'] = youtube_video_titles

print()
print("Done getting data.")

1 / 3094
2 / 3094
3 / 3094
4 / 3094
5 / 3094
6 / 3094
7 / 3094
8 / 3094
9 / 3094
10 / 3094
11 / 3094
12 / 3094
13 / 3094
14 / 3094
15 / 3094
16 / 3094
17 / 3094
18 / 3094
19 / 3094
20 / 3094
21 / 3094
22 / 3094
23 / 3094
24 / 3094
25 / 3094
26 / 3094
27 / 3094
28 / 3094
29 / 3094
30 / 3094
31 / 3094
32 / 3094
33 / 3094
34 / 3094
35 / 3094
36 / 3094
37 / 3094
38 / 3094
39 / 3094
40 / 3094
41 / 3094
42 / 3094
43 / 3094
44 / 3094
45 / 3094
46 / 3094
47 / 3094
48 / 3094
49 / 3094
50 / 3094
51 / 3094
52 / 3094
53 / 3094
54 / 3094
55 / 3094
56 / 3094
57 / 3094
58 / 3094
59 / 3094
60 / 3094
61 / 3094
62 / 3094
63 / 3094
64 / 3094
65 / 3094
66 / 3094
67 / 3094
68 / 3094
69 / 3094
70 / 3094
71 / 3094
72 / 3094
73 / 3094
74 / 3094
75 / 3094
76 / 3094
77 / 3094
78 / 3094
79 / 3094
80 / 3094
81 / 3094
82 / 3094
83 / 3094
84 / 3094
85 / 3094
86 / 3094
87 / 3094
88 / 3094
89 / 3094
90 / 3094
91 / 3094
92 / 3094
93 / 3094
94 / 3094
95 / 3094
96 / 3094
97 / 3094
98 / 3094
99 / 3094
100 / 3094
101 / 30

756 / 3094
757 / 3094
758 / 3094
759 / 3094
760 / 3094
761 / 3094
762 / 3094
763 / 3094
764 / 3094
765 / 3094
766 / 3094
767 / 3094
768 / 3094
769 / 3094
770 / 3094
771 / 3094
772 / 3094
773 / 3094
774 / 3094
775 / 3094
776 / 3094
777 / 3094
778 / 3094
779 / 3094
780 / 3094
781 / 3094
782 / 3094
783 / 3094
784 / 3094
785 / 3094
786 / 3094
787 / 3094
788 / 3094
789 / 3094
790 / 3094
791 / 3094
792 / 3094
793 / 3094
794 / 3094
795 / 3094
796 / 3094
797 / 3094
798 / 3094
799 / 3094
800 / 3094
801 / 3094
802 / 3094
803 / 3094
804 / 3094
805 / 3094
806 / 3094
807 / 3094
808 / 3094
809 / 3094
810 / 3094
811 / 3094
812 / 3094
813 / 3094
814 / 3094
815 / 3094
816 / 3094
817 / 3094
818 / 3094
819 / 3094
820 / 3094
821 / 3094
822 / 3094
823 / 3094
824 / 3094
825 / 3094
826 / 3094
827 / 3094
828 / 3094
829 / 3094
830 / 3094
831 / 3094
832 / 3094
833 / 3094
834 / 3094
835 / 3094
836 / 3094
837 / 3094
838 / 3094
839 / 3094
840 / 3094
841 / 3094
842 / 3094
843 / 3094
844 / 3094
845 / 3094
846 / 3094

1460 / 3094
1461 / 3094
1462 / 3094
1463 / 3094
1464 / 3094
1465 / 3094
1466 / 3094
1467 / 3094
1468 / 3094
1469 / 3094
1470 / 3094
1471 / 3094
1472 / 3094
1473 / 3094
1474 / 3094
1475 / 3094
1476 / 3094
1477 / 3094
1478 / 3094
1479 / 3094
1480 / 3094
1481 / 3094
1482 / 3094
1483 / 3094
1484 / 3094
1485 / 3094
1486 / 3094
1487 / 3094
1488 / 3094
1489 / 3094
1490 / 3094
1491 / 3094
1492 / 3094
1493 / 3094
1494 / 3094
1495 / 3094
1496 / 3094
1497 / 3094
1498 / 3094
1499 / 3094
1500 / 3094
1501 / 3094
1502 / 3094
1503 / 3094
1504 / 3094
1505 / 3094
1506 / 3094
1507 / 3094
1508 / 3094
1509 / 3094
1510 / 3094
1511 / 3094
1512 / 3094
1513 / 3094
1514 / 3094
1515 / 3094
1516 / 3094
1517 / 3094
1518 / 3094
1519 / 3094
1520 / 3094
1521 / 3094
1522 / 3094
1523 / 3094
1524 / 3094
1525 / 3094
1526 / 3094
1527 / 3094
1528 / 3094
1529 / 3094
1530 / 3094
1531 / 3094
1532 / 3094
1533 / 3094
1534 / 3094
1535 / 3094
1536 / 3094
1537 / 3094
1538 / 3094
1539 / 3094
1540 / 3094
1541 / 3094
1542 / 3094
1543

2143 / 3094
2144 / 3094
2145 / 3094
2146 / 3094
2147 / 3094
2148 / 3094
2149 / 3094
2150 / 3094
2151 / 3094
2152 / 3094
2153 / 3094
2154 / 3094
2155 / 3094
2156 / 3094
2157 / 3094
2158 / 3094
2159 / 3094
2160 / 3094
2161 / 3094
2162 / 3094
2163 / 3094
2164 / 3094
2165 / 3094
2166 / 3094
2167 / 3094
2168 / 3094
2169 / 3094
2170 / 3094
2171 / 3094
2172 / 3094
2173 / 3094
2174 / 3094
2175 / 3094
2176 / 3094
2177 / 3094
2178 / 3094
2179 / 3094
2180 / 3094
2181 / 3094
2182 / 3094
2183 / 3094
2184 / 3094
2185 / 3094
2186 / 3094
2187 / 3094
2188 / 3094
2189 / 3094
2190 / 3094
2191 / 3094
2192 / 3094
2193 / 3094
2194 / 3094
2195 / 3094
2196 / 3094
2197 / 3094
2198 / 3094
2199 / 3094
2200 / 3094
2201 / 3094
2202 / 3094
2203 / 3094
2204 / 3094
2205 / 3094
2206 / 3094
2207 / 3094
2208 / 3094
2209 / 3094
2210 / 3094
2211 / 3094
2212 / 3094
2213 / 3094
2214 / 3094
2215 / 3094
2216 / 3094
2217 / 3094
2218 / 3094
2219 / 3094
2220 / 3094
2221 / 3094
2222 / 3094
2223 / 3094
2224 / 3094
2225 / 3094
2226

2826 / 3094
2827 / 3094
2828 / 3094
2829 / 3094
2830 / 3094
2831 / 3094
2832 / 3094
2833 / 3094
2834 / 3094
2835 / 3094
2836 / 3094
2837 / 3094
2838 / 3094
2839 / 3094
2840 / 3094
2841 / 3094
2842 / 3094
2843 / 3094
2844 / 3094
2845 / 3094
2846 / 3094
2847 / 3094
2848 / 3094
2849 / 3094
2850 / 3094
2851 / 3094
2852 / 3094
2853 / 3094
2854 / 3094
2855 / 3094
2856 / 3094
2857 / 3094
2858 / 3094
2859 / 3094
2860 / 3094
2861 / 3094
2862 / 3094
2863 / 3094
2864 / 3094
2865 / 3094
2866 / 3094
2867 / 3094
2868 / 3094
2869 / 3094
2870 / 3094
2871 / 3094
2872 / 3094
2873 / 3094
2874 / 3094
2875 / 3094
2876 / 3094
2877 / 3094
2878 / 3094
2879 / 3094
2880 / 3094
2881 / 3094
2882 / 3094
2883 / 3094
2884 / 3094
2885 / 3094
2886 / 3094
2887 / 3094
2888 / 3094
2889 / 3094
2890 / 3094
2891 / 3094
2892 / 3094
2893 / 3094
2894 / 3094
2895 / 3094
2896 / 3094
2897 / 3094
2898 / 3094
2899 / 3094
2900 / 3094
2901 / 3094
2902 / 3094
2903 / 3094
2904 / 3094
2905 / 3094
2906 / 3094
2907 / 3094
2908 / 3094
2909

In [21]:
data.to_csv("data_try_algo_nn.csv", encoding="utf-8", header=True, index=False)